# 1.2.  Positions, Distances and Trajectories 


<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Licence" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" title='This work is licensed under a Creative Commons Attribution 4.0 International License.' align="right"/></a>

Authors: 

- Dr Micaela Matta - micaela.matta@kcl.ac.uk 
- Dr Richard Gowers - richardjgowers@gmail.com

This notebook is adapted from materials developed for the [2021 PRACE Workshop](https://github.com/MDAnalysis/WorkshopPrace2021) and the [2018 Workshop/Hackathon](https://github.com/MDAnalysis/WorkshopHackathon2018)


## Learning outcomes:


* How to access atom positions

* Calculating bonds and distances  

* Calculating angles

* How box information is stored and how this relates to periodic boundaries, wrapping/unwrapping 

* Efficient distance calculations and sparse arrays

* Loading trajectories, iterating over frames and selecting frames

### **Jupyter cheat sheet**:
- to run the currently highlighted cell, hold <kbd>&#x21E7; Shift</kbd> and press <kbd>&#x23ce; Enter</kbd>;
- to get help for a specific function, place the cursor within the function's brackets, hold <kbd>&#x21E7; Shift</kbd>, and press <kbd>&#x21E5; Tab</kbd>;

<div class="alert alert-warning"><b> REMEMBER: variables persist between cells</b> 
    
Be aware that it is the order of execution of cells that is important in a Jupyter notebook, not the <em>order</em> in which they appear. Python will remember <em>all</em> the code that was run previously, including any variables you have defined, irrespective of the order in the notebook. Therefore if you define variables lower down the notebook and then (re)run cells further up, those defined further down will still be present. </div> 

### **Additional resources**
 - During the workshop, feel free to ask questions at any time
 - For more on how to use MDAnalysis, see the [User Guide](https://userguide.mdanalysis.org/2.0.0-dev0/) and [documentation](https://docs.mdanalysis.org/2.0.0-dev0/)
 - Ask questions on the [user mailing list](https://groups.google.com/group/mdnalysis-discussion) or on [Discord](https://discord.gg/fXTSfDJyxE)
 - Report bugs on [GitHub](https://github.com/MDAnalysis/mdanalysis/issues?)


## Table of Contents

1. [Atom positions](#positions)  
2. [Bonds and Distances](#bonds)    
3. [Calculating Angles](#angles)     
4. [More distance calculations: distance arrays](#distarrays)
5. [Trajectories and Frames](#trajectory)

### Imports

We start with the usual imports:

In [1]:
import MDAnalysis as mda
import MDAnalysisData as data
import nglview

### Example: a PEG chain in water

Load a dataset from `MDAnalysisData`: a PEG - poly(ethyleneglycol) chain $HO(CH2CH2)_{20}OH$ in water

In [26]:
PEG_example = data.datasets.fetch_PEG_1chain()

Create a Universe by loading the topology and coordinates:

In [27]:
peg_u = mda.Universe(PEG_example['topology'], PEG_example['trajectory'])

We loaded a trajectory with 50 frames, but for the first part of this tutorial we'll limit ourselves to look at one frame.

In [ ]:
The polymer is the first residue:

In [28]:
peg = peg_u.atoms.residues[0]

### Visualize the system with `nglview`

`nglview` takes either a `Universe` or an `AtomGroup` as input:

In [29]:
poly = nglview.show_mdanalysis(peg)
poly

NGLWidget(max_frame=49)

## Atom positions
<a id='positions'></a>

The most important attribute of your atoms is undoubtedly their positions! The position information is made available via an `AtomGroup` in the `positions` attribute:

In [31]:
Os = peg_u.select_atoms('type os')

Os.positions

array([[17.474627, 29.40593 , 35.43572 ],
       [16.178513, 30.363686, 33.04827 ],
       [13.154764, 30.090767, 32.65325 ],
       [12.781218, 27.539545, 34.03944 ],
       [12.3174  , 24.909702, 35.999706],
       [15.073921, 23.41555 , 37.341427],
       [17.802116, 22.729006, 38.049328],
       [20.103138, 24.995148, 38.8918  ],
       [20.959639, 27.679546, 38.70003 ],
       [22.952808, 29.17524 , 37.071712],
       [25.204197, 28.613428, 34.926205],
       [26.116346, 27.930723, 32.045708],
       [28.322554, 25.94474 , 32.40819 ],
       [30.736938, 26.76612 , 30.284174],
       [30.406841, 27.816256, 27.381987],
       [29.278606, 27.836481, 23.95282 ],
       [28.370323, 26.439674, 21.368319],
       [31.034653, 25.007427, 20.5126  ],
       [33.20713 , 23.092627, 20.627098]], dtype=float32)

this returns a numpy array with x, y and z coordinates.

### Center of geometry

We can use `numpy` functions to manipulate coordinates. For instance, we can calculate the center of geometry:

In [53]:
cog = np.mean(Os.positions, axis=0)
print(cog)

[22.709246 26.829031 31.828304]


<div class="alert alert-info"> 
    
`AtomGroups` **and built-in functions** 

MDAnalysis `AtomGroups` have a convenient `center_of_mass()` method to get their center of mass at the current frame. 
Other  methods for common calculations based on positions include `center_of_geometry()`, `radius_of_gyration()` and `principal_axes()`. </div> 

In [54]:
Os.center_of_geometry()

array([22.70924909, 26.82903139, 31.82830459])

### Exercise: calculate radius of gyration of PEG

Using the `radius_of_gyration()` function, calculate the radius of gyration of PEG:

In [96]:
#select polymer as the first residue:
peg_chain = peg_u.residues[0]

rog = peg_chain.atoms.radius_of_gyration()


print(rog)

9.788402668965798


##  Bonds and Distances
<a id='bonds'></a>

### End-to-end distance of a PEG chain 

We can use the hydrogens in the capping -OH groups (`type ho`) as reference points. First, select the two hydrogen atoms:

In [34]:
Hb, He = peg_u.atoms.select_atoms("type ho")

In [38]:
np.linalg.norm(Hb.position - He.position)

26.629349

<div class="alert alert-info"> 
 
**built-in functions 2: bonds**
    
`MDAnalysis.lib.distances` has a `calc_bonds` method which allows users to calculate periodic aware distances between two sets of positions. </div> 

Then, calculate the distance between their coordinates:

In [37]:
mda.lib.distances.calc_bonds(Hb.position, He.position, box=peg_u.dimensions)

26.62934987068575

## Calculating Angles
<a id='angles'></a>

In [3]:
from MDAnalysisData.datasets import fetch_adk_transitions_DIMS
adk = fetch_adk_transitions_DIMS()
adk_u = mda.Universe(adk.topology, adk.trajectories[:1])

### AdK angles

The enzyme *adenylate kinase* catalyzes the reaction ATP + AMP <-> 2 ADP. 

It undergoes a *conformational transition* beteween a closed ([1AKE](https://www.rcsb.org/structure/1AKE)) and open ([4AKE](https://www.rcsb.org/structure/4AKE)) conformational state [1], even in the absence of substrates.

<center><img src="imgs/adk.png" alt="mda" style="width: 300px;"/></center>
1. S. L. Seyler and O. Beckstein. Sampling of large conformational transitions: Adenylate kinase as a testing ground. Molec. Simul., 40(10–11):855–877, 2014. doi: 10.1080/08927022.2014.919497

Let's look in more detail at the AdK protein. AdK has three domains:

 - CORE (residues 1-29, 60-121, 160-214)
 - NMP (residues 30-59)
 - LID (residues 122-159)

Angles between these domains can be used to distinguish open and closed states of the AdK protein. These angles are defined between the center of geometry of the backbone and C$_\beta$ atoms of the following groups of atoms:

 - $\theta_{NMP}$ is defined between residues:
   - A: 115-125 
   - B: 90-100
   - C: 35-55
 - $\theta_{LIC}$ is defined between residues:
   - A: 179-185
   - B: 112-125
   - C: 125-153 



The angle between two vectors is given by:

$$\theta = arccos\left( \frac{\vec{BA}\cdot\vec{BC}}{|\vec{BA}||\vec{BC}|} \right)$$

We first define the coordinate centers for each residue by using `center_of_geometry`:

In [9]:
A_NMP = adk_u.select_atoms('resid 115-125 and (backbone or name CB)').center_of_geometry()
B_NMP = adk_u.select_atoms('resid 90-100 and (backbone or name CB)').center_of_geometry()
C_NMP = adk_u.select_atoms('resid 35-55 and (backbone or name CB)').center_of_geometry()

A_LID = adk_u.select_atoms('resid 179-185 and (backbone or name CB)').center_of_geometry()
B_LID = adk_u.select_atoms('resid 112-125 and (backbone or name CB)').center_of_geometry()
C_LID = adk_u.select_atoms('resid 125-153 and (backbone or name CB)').center_of_geometry()

### the Numpy way 


You can use `numpy.linalg.norm()` to calculate the norm of a vector. Numpy also has functions `numpy.arccos()` and `numpy.dot()`:

In [7]:
import numpy as np
from numpy.linalg import norm

# define vectors BA and BC
BA_NMP = A_NMP - B_NMP
BC_NMP = C_NMP - B_NMP

# calculate theta_NMP
theta_NMP = np.arccos(np.dot(BA_NMP, BC_NMP)/(norm(BA_NMP)*norm(BC_NMP)))

print('theta_NMP: ', np.rad2deg(theta_NMP))

# define vectors BA and BC
BA_LID = A_LID - B_LID
BC_LID = C_LID - B_LID

# calculate theta_LID
theta_LID = np.arccos(np.dot(BA_LID, BC_LID)/(norm(BA_LID)*norm(BC_LID)))

print('theta_LID: ', np.rad2deg(theta_LID))

theta_NMP:  43.71472676451854
theta_LID:  106.4094929404029


<div class="alert alert-info"> 
 
**built-in functions 4: angles and dihedrals**
    
`MDAnalysis.lib.distances` has a `calc_angles` method which allows users to calculate periodic aware angles between 3 array-like sets of positions. 
    
In the same way, it is possible to calculate dihedrals with `calc_dihedrals`. </div> 

### the  `MDAnalysis` way

There's an alternative, much faster way! 
Let's use `calc_angles` to get the angles $\theta_{NMP}$ and $\theta_{LID}$:

In [8]:
theta_NMP = mda.lib.distances.calc_angles(A_NMP, B_NMP, C_NMP)
theta_LID = mda.lib.distances.calc_angles(A_LID, B_LID, C_LID)

print('theta_NMP: ', np.rad2deg(theta_NMP))
print('theta_LID: ', np.rad2deg(theta_LID))

theta_NMP:  43.7147272459085
theta_LID:  106.40949206629575


`calc_angles`, like all `MDAnalysis.lib.distances` functions, has an optional argument to specify periodic boundary conditions:
```
 mda.lib.distances.calc_angles(
    coords1,
    coords2,
    coords3,
    box=None,
    result=None,
    backend='serial')
   ``` 
    

## More distances and angles: hydrogen bonds
<a id='distarrays'></a>

This example will show you how to use various functions in `MDAnalysis.lib.distances` to identify hydrogen bonding between certain residues and the water solvent.

A hydrogen bond (in the context of this analysis) will be defined as an interaction between three atoms:
- An acceptor, which is attracting the hydrogen
- A hydrogen, which is being pulled into the acceptor
- A donor, which is bonded to the hydrogen and being dragged along for the ride.

We will use the following geometric criteria:
- a hydrogen-acceptor distance of 3.0A 
- an acceptor-hydrogen-donor angle of greater than 120 degrees.

We can go back to our PEG chain in water. Oxygen atoms in PEG can accept hydrogen bonds from water:

In [12]:
# select oxygen atoms - types os and oh
acceptors = peg_u.atoms.select_atoms("type os oh")

Select hydrogens (from water):

In [39]:
hydrogens = peg_u.atoms.select_atoms("type HW")

### Distance criteria

We first want to identify hydrogens and acceptors that are within our distance criteria of 3.0 angstrom.
A naive approach is to calculate a `distance_array` between all acceptors and all hydrogens.:

In [40]:
%%time

da = mda.lib.distances.distance_array(acceptors.positions, hydrogens.positions, box=peg_u.dimensions)

CPU times: user 7.09 ms, sys: 1.45 ms, total: 8.54 ms
Wall time: 7.16 ms


<div class="alert alert-info"> 

**Hint:** `np.where` is a handy function for returning the *indices* of where a condition is True.  Here we use it to extract the row and column numbers of where an entry in a distance matrix is less than 3.0.</div>
     

In [41]:
acc_idx, hyd_idx = np.where(da < 3.0)

### Using `capped_distance`

This is a great example of where we're not interested in all distances, but instead only those up to a given cutoff - Using `capped_distance` is much quicker here!

<div class="alert alert-info"> 

**Reminder:** The output of `capped_distance` is no longer a matrix, but an array of indices and the distance values at those indices.  This can be thought of as a sparse matrix.
 </div>

Try experimenting with the cutoff distance to see how the time required varies.

In [55]:
%%time 

idx, dists = mda.lib.distances.capped_distance(acceptors.positions, hydrogens.positions, max_cutoff=3.0,
                                            box=peg_u.dimensions)

CPU times: user 1.24 ms, sys: 646 µs, total: 1.88 ms
Wall time: 1.02 ms


The `idx` array is a `(n, 2)` array of indices; to grab the first and second column, we can transpose the array (`.T`) and assign each row to a varaible, `acc_idx` for the *indices* of the acceptors and `hyd_idx` for the *indices* of the hydrogen atoms.

In [43]:
acc_idx, hyd_idx = idx.T

Remembering that we can slice `AtomGroup`s with numpy arrays, we can use these indices arrays to slice our original `AtomGroup`s to filter them down and make them smaller.

In [44]:
# select potential hydrogen bonds to check angles
potential_hbond_acceptors = acceptors[acc_idx]
potential_hbond_hydrogens = hydrogens[hyd_idx]

To get the **donors** for each hydrogen bond is slightly trickier.
We can use the fact that hydrogens will only have one covalent bond, and simply loop over the hydrogen atoms, grabbing the first (and only) bonded atom of each. 

<div class="alert alert-info"> 

**Reminder:** `sum()` over `MDAnalysis.Atom` objects will produce an `AtomGroup`!

</div>

In [45]:
potential_hbond_donors = sum(h.bonded_atoms[0] for h in potential_hbond_hydrogens)

## Checking the angle criteria

Now that we've identified hydrogens and acceptors which are close enough for a hydrogen bond, we can now check our angular criteria.
The angle formed by the acceptor-hydrogen-donor must be greater than 120 degrees!


<div class="alert alert-info"> 
    
**Reminder**: The input to `calc_angles` must be in the correct order, with the second array of positions being the vertex of the angle.  Results are returned in radians!
 </div>

By first checking the distance criteria and filtering down our input, we greatly reduce the number of angles we must calculate.
This is important as angles calculations are computationally more expensive than distance calculations.

In [48]:
angles = np.rad2deg(
    mda.lib.distances.calc_angles(potential_hbond_acceptors.positions,
                                  potential_hbond_hydrogens.positions,
                                  potential_hbond_donors.positions, box=peg_u.dimensions)
)

Again we can use `np.where` to get the *indices* of where a condition is True, here if a value is above 120.

In [49]:
angle_idx = np.where(angles >= 120.0)

Finally, we can slice our list of candidate atoms with `angle_idx` to get three `AtomGroup`s, each representing a different component in a hydrogen bond.

In [50]:
hbond_acceptors = potential_hbond_acceptors[angle_idx]
hbond_hydrogens = potential_hbond_hydrogens[angle_idx]
hbond_donors = potential_hbond_donors[angle_idx]

In [51]:
hbond_donors

<AtomGroup with 69 atoms>

### The Analysis class way

Because hydrogen bond analysis is so common, it already exists as an Analysis class:

In [58]:
from MDAnalysis.analysis.hydrogenbonds import HydrogenBondAnalysis

In [67]:
hbonds = HydrogenBondAnalysis(peg_u,
                             acceptors_sel='type os oh',
                             hydrogens_sel='type HW')

We can then run analysis for the first 5 frames of the trajectory:

In [76]:
hbonds.run(stop=5, verbose=True)

  0%|          | 0/5 [00:00<?, ?it/s]

In [84]:
hbonds.results['hbonds']

array([[0.00000000e+00, 3.26000000e+02, 3.27000000e+02, 9.10000000e+01,
        2.63570308e+00, 1.55290512e+02],
       [0.00000000e+00, 1.05800000e+03, 1.05900000e+03, 1.19000000e+02,
        2.83754533e+00, 1.60064259e+02],
       [0.00000000e+00, 1.27400000e+03, 1.27500000e+03, 1.50000000e+01,
        2.75470781e+00, 1.52915222e+02],
       [0.00000000e+00, 1.47800000e+03, 1.48000000e+03, 9.90000000e+01,
        2.54154955e+00, 1.72443505e+02],
       [0.00000000e+00, 2.32400000e+03, 2.32600000e+03, 1.33000000e+02,
        2.79760614e+00, 1.70921065e+02],
       [0.00000000e+00, 2.44700000e+03, 2.44900000e+03, 7.00000000e+01,
        2.66524897e+00, 1.54749772e+02],
       [0.00000000e+00, 3.22400000e+03, 3.22500000e+03, 1.39000000e+02,
        2.82700029e+00, 1.68624274e+02],
       [0.00000000e+00, 3.34100000e+03, 3.34200000e+03, 2.70000000e+01,
        2.88533806e+00, 1.72663787e+02],
       [0.00000000e+00, 3.34100000e+03, 3.34300000e+03, 7.00000000e+00,
        2.94131797e+00, 

limit_output extension: Maximum message size of 10000 exceeded with 11865 characters

<div class="alert alert-info"> 
    
**Reminder:** By default all frames will be analysed, defining `start`, `stop`, `step` in `run()` will control how the trajectory is sliced.
    
   </div>

<div class="alert alert-info"> 

#### RECAP: how to calculate quickly all possible distances between `AtomGroups` 

 - `self_distance` array only takes one atomgroup
 - `distance_array` takes two atomgroups and they don't have to contain the same number of atoms.
 - `capped_distance` and `self_capped_distance` only considers atoms within a certain distance threshold.
 </div>

### Further work


- A radial distribution function can be calculated using a histogram of distances.  Using `np.histogram` (to make a histogram), how could you calculate the distribution of distances between two AtomGroups? Plot the result. Is the method required any different over a trajectory than a frame?

This is an open-ended question so we do not provide a solution. Please feel free to reach out to any of the above resources (in the first cell) for more help.

## Trajectories and Frames
<a id='trajectory'></a>


### Loading a trajectory
Loading a trajectory is done in the same way as loading any type of coordinates (as shown in session 1). All you have to do is create a `Universe` object by passing it a topology and the trajectory (here in this case a PSF file and DCD trajectory respectively).

In [85]:
# First let's load a PSF and DCD from the MDAnalysis test data
from MDAnalysis.tests.datafiles import PSF, DCD
u = mda.Universe(PSF, DCD)

Trajectory functionality is centered around the `Universe.trajectory` object.

In [86]:
u.trajectory

<DCDReader /Users/micaela/anaconda3/envs/mda/lib/python3.7/site-packages/MDAnalysisTests/data/adk_dims.dcd with 98 frames of 3341 atoms>

This `trajectory` object has a length in `frames` and a time unit of **picoseconds** (more information about the [MDAnalysis base units](https://docs.mdanalysis.org/2.0.0-dev0/documentation_pages/units.html#id4) is in the docs).

The `trajectory` object has many useful attributes, such as the the number of frames `n_frames`, the time between frames `dt`, the total trajectory time `totaltime`.

In [87]:
# print the number of frames
u.trajectory.n_frames

98

In [88]:
# You can also get the number of frames by calling `len` on the trajectory object
len(u.trajectory)

98

In [89]:
# We can get the time between frames with `dt`
u.trajectory.dt

0.9999999119200186

In [90]:
# And the total simulation time from `totaltime`
u.trajectory.totaltime

96.9999914562418

### The Timestep object

One of the key components of trajectories is the *Timestep* object `ts`. This is the object that holds the trajectory information **specific to the current frame**.

This information mainly includes:
* The frame number and time
* Unitcell dimensions as `[A, B, C, alpha, beta, gamma]` (or `None` if not available)
* The positions (also forces and/or velocities if available)

In [91]:
u.trajectory.ts

< Timestep 0 with unit cell dimensions None >

### Moving through a trajectory

Up until this point, we have primarily been inspecting only a single frame of the `trajectory` object. By default when creating a `Universe`, the *Timestep* is loaded with the information from the first (zero-th) frame in the trajectory.

Here we look at how we can traverse through the trajectory and access the data from different frames.

We can consider the `trajectory` object to be an iterator that loads trajectory data from a source (i.e. in most cases the input trajectory file), and feeds the relevant data to the *Timestep* object.

The following operations can be done to access the trajectory:
* Random access via trajectory indexing
* Iterating over all frames
* Slicing to iterate over a sub-section of the trajectory


<div class="alert alert-info"> 

#### NOTE:
 
As is standard in python, `trajectory` access is done via **0-based indices**. So the first frame is `0`, and the final frame is `n_frames - 1`. 
</div>

### Trajectory indexing

It is possible to randomly access any frame along a trajectory by passing the index of the frame to the trajectory.

In [ ]:
# Let's create an atomgroup for the first two atoms in the Universe
# and check their current position at frame 0
first_two_atoms = u.atoms[:2]
print(first_two_atoms.positions)

In [ ]:
# Now let's move to the 7th frame
u.trajectory[6]

In [ ]:
# As we can see the frame number as now updated accordingly
print('current frame: ', u.trajectory.frame)

In [ ]:
# The AtomGroup also automatically updates with the new Timestep data
print(first_two_atoms.positions)

<div class="alert alert-info">
    
#### NOTE: `AtomGroup`s are not static objects!
    
Whilst the atoms they represent do not change (see `UpdatingAtomGroup` for when this is not the case), their positions (and forces or velocities if available) will change as you move through the trajectory.

</div>

any changes to variables that change with `Timestep`, are *temporary*.

For example, if you were to override the position of an `AtomGroup` for a given frame, then seek to another frame and come back to the original frame, the positions would be updated back to reflect the contents of the trajectory file:

In [ ]:
# Let's start from frame 0 and override the positions of `first_two_atoms`
u.trajectory[0]

# `first_two_atoms` positions beforehand
print('frame 0 positions: ', first_two_atoms.positions)

# `first_two_atoms` after being zeroed
first_two_atoms.positions = 0
print('zeroed positions: ', first_two_atoms.positions)

In [ ]:
# Now let's go the second last frame
u.trajectory[-2]
first_two_atoms.positions

In [ ]:
# And now we come back to frame 0
u.trajectory[0]

# positions are no longer zeroed
first_two_atoms.positions

### Exercise 

Run these two assertions in a separate cell. Is the output what you expect? Why or why not?

```python
print("Equality", first_two_atoms.positions == first_two_atoms.positions)
print("Same object in memory", first_two_atoms.positions is first_two_atoms.positions)
```

What consequences might arise? e.g. if you wanted to set all Z positions to 0, is the below statement feasible?

```python
first_two_atoms.positions[:, 2] = 0
print(first_two_atoms.positions)
```

How might you get around this?

In [ ]:
# Exercise 

Running the first pair of statements gives a perplexing result.

```ipython
>>> print("Equality", first_two_atoms.positions == first_two_atoms.positions)
Equality [[ True  True  True]
 [ True  True  True]]
>>> print("Same object in memory", first_two_atoms.positions is first_two_atoms.positions)
Equality False
```

Even though MDAnalysis makes it look as though the `positions` array is a static property that belongs to `first_two_atoms`, remember that the fundamental object that stores dynamic data such as `positions` is the *Timestep*. This means that every time you ask an AtomGroup for its `positions`, the AtomGroup actually asks the Timestep for the positions of *all* the atoms, and then only returns the ones for that atoms inside the group. Therefore, every time you call `AtomGroup.positions`, you are returned a new array.

This also explains why directly changing the Z coordinates of `AtomGroup.positions` does not seem to have any effect -- you have modified the values of the the returned copy, not the underlying positions array stored in `Timestep`. So how do you modify the coordinates of an AtomGroup? Well, *setting* the `AtomGroup.positions` attribute will transmit the change to the underlying Timestep. So, the general construct should be:

```python
>>> coordinates = first_two_atoms.positions
>>> coordinates[:, 2] = 0
>>> first_two_atoms.positions = coordinates
>>> print(first_two_atoms.positions)
[[15.249873 12.578178  0.      ]
 [14.925511 13.58888   0.      ]]
```

But remember, these changed coordinates will be gone when you switch the active frame, unless the trajectory is in memory.

In [ ]:
# Exercise 

### Iterating through the trajectory

Iterating through a trajectory is the most common way to move through a trajectory.

For example one could access every frame in the trajectory and store the current time using the following:

In [ ]:
# Create a list for the times
times = []

for ts in u.trajectory:
    times.append(u.trajectory.time)
    
print(times)

### Trajectory slicing

Rather than iterating through the entire trajectory, it is possible to slice the trajectory using a `[start:stop:step]` pattern.

In [ ]:
# Let's slice starting at the second frame, ending on the before last frame
# and skipping every other frame

times = []

for ts in u.trajectory[1:-2:2]:
    times.append(u.trajectory.time)
    
print(times)

### Exercise

Create a reversed list of the trajectory times

In [ ]:
# Exercise 4 -- solution
times = []

for ts in u.trajectory[::-1]:
    times.append(u.trajectory.time)
    
print(times)